# Listas de Filas Trabalho

Hyago Vieira



1) Carros entram em uma fila de pedágio de acordo com um processo de Poisson de
taxa 3 carros a cada cinco minutos, o tempo de atendimento segue uma variável
exponencial de média 1/μ = 1 minuto.

a) Qual é o tempo médio dos carros no sistema?

b) Qual é o número médio de carros na fila?


In [6]:
import numpy as np

# Parâmetros do sistema: estado da fila e do servidor, taxas de chegada e de serviço
estado_fila, estado_servidor = [0, 1]
taxa_chegada, taxa_servico = [3/5, 1]

# Inicialização dos tempos de chegada e partida, usando distribuição exponencial
tempo_chegada = np.random.exponential(1/taxa_chegada, 1)
tempo_partida = np.random.exponential(1/taxa_servico, 1)
tempos_chegada, tempos_partida, tempos_sistema = [[0], [], []]
proximo_evento = np.minimum(tempo_chegada, tempo_partida)
num_partidas = 0

# Loop principal de simulação
while proximo_evento < 20000:
    if tempo_chegada < tempo_partida:  # Evento de chegada
        tempos_chegada = np.append(tempos_chegada, proximo_evento)
        tempo_chegada = proximo_evento + np.random.exponential(1/taxa_chegada, 1)

        if estado_servidor == 1:
            estado_fila += 1  # Incrementa elementos na fila
        else:
            estado_servidor = 1
            tempo_partida = proximo_evento + np.random.exponential(1/taxa_servico, 1)

    else:  # Evento de partida
        num_partidas += 1
        tempos_partida = np.append(tempos_partida, proximo_evento)

        if estado_fila > 0:
            estado_fila -= 1
            tempo_partida = proximo_evento + np.random.exponential(1/taxa_servico, 1)
        else:
            estado_servidor = 0
            tempo_partida = np.inf

        tempos_sistema = np.append(tempos_sistema, tempos_partida[num_partidas-1] - tempos_chegada[num_partidas-1])

    proximo_evento = np.minimum(tempo_chegada, tempo_partida)

# Cálculo das métricas de desempenho do sistema
tempo_medio_fila = np.mean(tempos_sistema) - 1/taxa_servico
tempo_medio_sistema = np.mean(tempos_sistema)

print(f'Número médio de elementos na fila = {taxa_chegada * tempo_medio_fila:.4f} carros.')
print(f'Tempo médio no sistema = {tempo_medio_sistema:.4f} minutos.')


Número médio de elementos na fila = 0.9108 carros.
Tempo médio no sistema = 2.5179 minutos.


2) Um comutador de pacotes possui uma linha de saída e recebe, em média, 40
pacotes por segundo. Cada pacote tem, em média, 5.000 bits de comprimento,
com distribuição exponencial. A linha de saída do comutador tem taxa de 500
kbps.

a) Qual é o tempo médio de permanência de um pacote no comutador (esperando
na fila e sendo transmitido)?

b) Qual é o tempo médio de espera na fila?

In [13]:
import numpy as np

num_fila, estado_servidor = [0, 1]       # Elementos na Fila e no Servidor.
taxa_chegada, taxa_servico = [40, 1/0.01]  # Taxas de chegada e de atendimento.

# Próximos chegada, partida e tempo de execução para atendimento exponencial:
tempo_chegada, tempo_partida = np.random.exponential(1/taxa_chegada, 1), np.random.exponential(1/taxa_servico, 1)
tempos_chegadas, tempos_partidas, tempos_sistema = [[0], [], []]
tempo_evento = np.minimum(tempo_chegada, tempo_partida)
num_partidas = 0             # Número de partidas.

while (tempo_evento < 1000):
  if tempo_chegada < tempo_partida:     # Próximo evento é uma chegada
    tempos_chegadas = np.append(tempos_chegadas, tempo_evento)
    tempo_chegada = tempo_evento + np.random.exponential(1/taxa_chegada, 1)  # Gera nova chegada.

    if estado_servidor == 1: num_fila = num_fila + 1   # Aumenta o número de elementos na fila.
    else:                     # Se o servidor estiver ocioso:
      estado_servidor = 1                  # Ocupa o servidor livre e gera a partida.
      tempo_partida = tempo_evento + np.random.exponential(1/taxa_servico, 1)

  else:         # Próximo evento é uma partida
    num_partidas = num_partidas + 1
    tempos_partidas = np.append(tempos_partidas, tempo_evento)

    if num_fila > 0:  # Se existem elementos na fila:
      num_fila = num_fila - 1 # Diminui o número de elementos na fila e gera a partida.
      tempo_partida = tempo_evento + np.random.exponential(1/taxa_servico, 1)
    else:       # Se não há pacotes na fila:
      estado_servidor = 0    # Servidor vazio
      tempo_partida = np.infty

    tempos_sistema = np.append(tempos_sistema, (tempos_partidas[num_partidas-1] - tempos_chegadas[num_partidas-1]))
  tempo_evento = np.minimum(tempo_chegada, tempo_partida) # Ocorrência do próximo evento.

tempo_medio_fila, tempo_medio_sistema = [np.mean(tempos_sistema) - 1/taxa_servico, np.mean(tempos_sistema)]
print(f'Tempo Médio de espera na Fila = {tempo_medio_fila*1000:.4f} ms')
print(f'Tempo médio no Sistema = {tempo_medio_sistema*1000:.4f} ms')


Tempo Médio de espera na Fila = 6.6463 ms
Tempo médio no Sistema = 16.6463 ms


3) Um comutador de pacotes recebe em média 200 pacotes/segundo, cada um com
um comprimento médio de 128 bytes. O comutador possui uma única linha de
saída com capacidade de 256 kbps. Considere um buffer com {1,5,10,15}
posições na fila, qual a probabilidade de bloqueio, número médio de elementos e
tempo médio no sistema?

In [12]:
for buffer_capacidade in [1, 5, 10, 15]:  # Capacidade do buffer
  tempo, partidas = [0, 0]                # Tempo inicial e número de partidas
  num_fila, num_servidor = [0, 1]         # Número de elementos na fila e no servidor
  taxa_chegada, taxa_atendimento = [200, 250]  # Taxa média de chegada e atendimento
  num_bloqueios, total_chegadas = [0, 0]  # Número de elementos bloqueados e total de chegadas

  tempo_chegada = tempo + np.random.exponential(1/taxa_chegada, 1)  # Tempo inicial de chegada
  tempo_partida = tempo + np.random.exponential(1/taxa_atendimento, 1)  # Tempo inicial de partida
  tempo = np.minimum(tempo_chegada, tempo_partida)  # Próximo evento

  # Tempos de chegada, partida e de permanência no sistema:
  tempos_chegada, tempos_partida, tempos_sistema = [[0], [], []]
  while (tempo < 1000):
    if tempo_chegada < tempo_partida:  # Próximo evento é uma chegada
      total_chegadas += 1                             # Contabiliza uma chegada
      tempo_chegada = tempo + np.random.exponential(1/taxa_chegada, 1)  # Gera próxima chegada

      if num_servidor == 1:  # Se o servidor estiver ocupado
        if num_fila == buffer_capacidade:
          num_bloqueios += 1  # Bloqueio se o buffer estiver cheio
        else:  # Armazena o elemento no buffer
          num_fila += 1  # Incrementa o número de elementos na fila
          tempos_chegada = np.append(tempos_chegada, tempo)
      else:  # Se o servidor estiver livre
        num_servidor = 1  # Ocupa o servidor livre
        tempo_partida = tempo + np.random.exponential(1/taxa_atendimento, 1)  # Gera próxima partida
        tempos_chegada = np.append(tempos_chegada, tempo)
    else:  # Próximo evento é uma partida
      partidas += 1
      tempos_partida = np.append(tempos_partida, tempo)

      if num_fila > 0:  # Se existem elementos na fila
        num_fila -= 1  # Decrementa a fila
        tempo_partida = tempo + np.random.exponential(1/taxa_atendimento, 1)  # Gera próxima partida
      else:
        num_servidor = 0
        tempo_partida = np.infty

      tempos_sistema = np.append(tempos_sistema, (tempos_partida[partidas-1] - tempos_chegada[partidas-1]))
    tempo = np.minimum(tempo_chegada, tempo_partida)

  utilizacao_servidor, prob_bloqueio = [taxa_chegada/taxa_atendimento, num_bloqueios/total_chegadas]
  tempo_espera, tempo_fila = [np.mean(tempos_sistema) - 1/taxa_atendimento, np.mean(tempos_sistema)]
  media_espera, media_fila = [taxa_chegada*tempo_espera*(1 - prob_bloqueio), taxa_chegada*tempo_fila*(1 - prob_bloqueio)]

  print(f'Para sistema com capacidade do buffer de {buffer_capacidade}:')
  print(f'Número e tempo médio de elementos no sistema = {media_fila:.4f}, {tempo_fila*1000:.4f} ms')
  print(f'Probabilidade de bloqueio = {prob_bloqueio:.4f}\n')


Para sistema com capacidade do buffer de 1:
Número e tempo médio de elementos no sistema = 0.8496, 5.7460 ms
Probabilidade de bloqueio = 0.2607

Para sistema com capacidade do buffer de 5:
Número e tempo médio de elementos no sistema = 2.1230, 11.3531 ms
Probabilidade de bloqueio = 0.0650

Para sistema com capacidade do buffer de 10:
Número e tempo médio de elementos no sistema = 3.0893, 15.7325 ms
Probabilidade de bloqueio = 0.0182

Para sistema com capacidade do buffer de 15:
Número e tempo médio de elementos no sistema = 3.6220, 18.2174 ms
Probabilidade de bloqueio = 0.0059



4) Um nó de uma rede de computadores possui buffer infinito. A chegada das
mensagens é Poissoniana com taxa 1 mensagem/segundo e o tamanho médio das
mensagens é igual a 2.000 bits. A capacidade do meio de transmissão é de
10.000 bps. Determine o tempo médio que uma mensagem permanece no nó
(espera + serviço) supondo que o comprimento das mensagens:

a) é constante.

b) tem distribuição exponencial.

In [15]:
for distribuicao in ['constante', 'exponencial']:
  fila, servidor = [0, 1]  # Elementos na fila e no servidor
  taxa_chegada, taxa_atendimento = [1, 5]  # Taxas de chegada e atendimento

  # Próximos eventos de chegada, partida e tempo de execução dependem da distribuição
  # do tamanho dos pacotes:
  tempo_chegada = np.random.exponential(1/taxa_chegada, 1)
  tempo_partida = np.random.exponential(1/taxa_atendimento, 1) if distribuicao == 'exponencial' else 1/taxa_atendimento
  tempos_chegadas, tempos_partidas, tempos_sistema = [[0], [], []]
  tempo = np.minimum(tempo_chegada, tempo_partida)
  partidas = 0  # Número de partidas

  while (tempo < 50000):
    if tempo_chegada < tempo_partida:  # Próximo evento é uma chegada
      tempos_chegadas = np.append(tempos_chegadas, tempo)
      tempo_chegada = tempo + np.random.exponential(1/taxa_chegada, 1)  # Gera nova chegada

      if servidor == 1:
        fila += 1  # Aumenta o número de elementos na fila
      else:  # Se o servidor estiver ocioso:
        servidor = 1  # Ocupa o servidor livre e gera a partida
        tempo_partida = tempo + (np.random.exponential(1/taxa_atendimento, 1) if distribuicao == 'exponencial' else 1/taxa_atendimento)

    else:  # Próximo evento é uma partida
      partidas += 1
      tempos_partidas = np.append(tempos_partidas, tempo)

      if fila > 0:  # Se existem elementos na fila:
        fila -= 1  # Diminui o número de elementos na fila e gera a partida
        tempo_partida = tempo + (np.random.exponential(1/taxa_atendimento, 1) if distribuicao == 'exponencial' else 1/taxa_atendimento)
      else:  # Se não há pacotes na fila:
        servidor = 0  # Servidor vazio
        tempo_partida = np.infty

      tempos_sistema = np.append(tempos_sistema, (tempos_partidas[partidas-1] - tempos_chegadas[partidas-1]))
    tempo = np.minimum(tempo_chegada, tempo_partida)  # Ocorrência do próximo evento

  print(f'Sistema com distribuição {distribuicao}:')
  print(f'Tempo médio no Sistema = {np.mean(tempos_sistema)*1000:.4f} ms.\n')


Sistema com distribuição constante:
Tempo médio no Sistema = 225.3621 ms.

Sistema com distribuição exponencial:
Tempo médio no Sistema = 250.8108 ms.

